# 梯度下降法的调试

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
np.random.seed(666)
X = np.random.random(size=(1000, 10))

In [3]:
true_theta = np.arange(1, 12, dtype=float) # 1到12，步长为1，左闭右开，总共11个元素

In [4]:
X_b = np.hstack([np.ones((len(X), 1)), X])
y = X_b.dot(true_theta) + np.random.normal(size=1000)

In [5]:
X.shape

(1000, 10)

In [6]:
X_b.shape # X_b的列等于true_theta的行，X_b.dot(true_theta)时，true_theta会自动转成11行1列的一维向量

(1000, 11)

In [7]:
'''多项式的系数，也即最终要拟合出来的结果'''
true_theta

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

## J和dJ的表达式
> 下面的图源自6.3
![最终的损失函数表达式](../03-Gradient-Descent-In-Regression/images/最终的损失函数表达式.png)
> y_hat的推导见[5.7多元线性回归和正规方程解](https://gitee.com/lsgwr/algorithms/blob/master/Part5Improve/05-Linear-Regression/07-Multi-Linear-Regression/多元线性回归和正规方程解.md)
![y_hat的推导](images/y_hat的推导.png)

In [9]:
def J(theta, X_b, y):
    try:
        return np.sum((y - X_b.dot(theta))**2) / len(X_b)
    except:
        return float('inf')

In [10]:
def dJ_math(theta, X_b, y):
    return X_b.T.dot(X_b.dot(theta) - y) * 2. / len(y)

In [11]:
def gradient_descent(dJ, X_b, y, initial_theta, eta, n_iters = 1e4, epsilon=1e-8): # 这里的第一个参数dJ是我们自己定义的，比如dJ_math和dJ_debug
    
    theta = initial_theta
    cur_iter = 0

    while cur_iter < n_iters:
        gradient = dJ(theta, X_b, y)
        last_theta = theta
        theta = theta - eta * gradient
        if(abs(J(theta, X_b, y) - J(last_theta, X_b, y)) < epsilon):
            break
            
        cur_iter += 1

    return theta

## 带调试的梯度下降法的推导
![梯度调试公司推导1](images/梯度调试公司推导1.png)
![梯度调试公司推导2](images/梯度调试公司推导2.png)
![梯度调试公司推导3](images/梯度调试公司推导3.png)

In [12]:
def dJ_debug(theta, X_b, y, epsilon=0.01): # 非向量运算，要比dJ_math慢很多
    res = np.empty(len(theta))
    for i in range(len(theta)):
        theta_1 = theta.copy()
        theta_1[i] += epsilon
        theta_2 = theta.copy()
        theta_2[i] -= epsilon
        res[i] = (J(theta_1, X_b, y) - J(theta_2, X_b, y)) / (2 * epsilon)
    return res

In [14]:
'''带调试的梯度下降法'''
X_b = np.hstack([np.ones((len(X), 1)), X])
initial_theta = np.zeros(X_b.shape[1]) # theta的长度等于X_b的列数
eta = 0.01

%time theta = gradient_descent(dJ_debug, X_b, y, initial_theta, eta)
theta

CPU times: user 8.13 s, sys: 6.19 s, total: 14.3 s
Wall time: 7.24 s


array([  1.1251597 ,   2.05312521,   2.91522497,   4.11895968,
         5.05002117,   5.90494046,   6.97383745,   8.00088367,
         8.86213468,   9.98608331,  10.90529198])

In [15]:
''' 数学推导的梯度下降法(不调试的情况下，速度要快很多，一半先用debug方法dj_debug获取正确结果，然后用数学推导法dJ_math去验证)'''
%time theta = gradient_descent(dJ_math, X_b, y, initial_theta, eta)
theta

CPU times: user 950 ms, sys: 660 ms, total: 1.61 s
Wall time: 835 ms


array([  1.1251597 ,   2.05312521,   2.91522497,   4.11895968,
         5.05002117,   5.90494046,   6.97383745,   8.00088367,
         8.86213468,   9.98608331,  10.90529198])